In [97]:
import pandas as pd
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [98]:
def crop_frames(video_path):
  video=cv2.VideoCapture(video_path)
  frame_array=[]
  fps = video.get(cv2.CAP_PROP_FPS)
  frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
  duration = frame_count/fps
  _,img=video.read()
  while(_):
    if _ is None:
      break
    cropped_img=img[180:750,400:1380] # These neeeded to be changed manually based on video
    frame_array.append(cropped_img)
    _,img=video.read()
  
  return frame_array,fps,frame_count,duration

In [99]:
def display_all_frames(frame_array):
  for i in range(len(frame_array)):
    plt.imshow(frame_array[i])
    plt.show()

In [100]:
def diff_btwn_adjacent_frames(frames_array):
  diff_array=[]
  dimensions=[]
  for i in range(1,len(frames_array)):
    frame1=frames_array[i-1].copy()
    frame2=frames_array[i].copy()
    diff=cv2.absdiff(frame1,frame2)
    _,thresh=cv2.threshold(diff,140,250,cv2.THRESH_BINARY)
    gray=cv2.cvtColor(thresh,cv2.COLOR_BGR2GRAY)
    #ada_thresh=cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
    kernel=np.ones((2,2),np.uint8)
    dilation=cv2.dilate(gray,kernel,2)
    thresh=cv2.adaptiveThreshold(dilation,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
    contours,hierarchy=cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    if(len(contours)>0):
      for contour in contours:
        area=cv2.contourArea(contour)
        if area<200 or area>900:
          continue
        x,y,w,h=cv2.boundingRect(contour)
        dimensions.append([x,y,w,h,i])
        color=np.random.uniform((255,255,255))
        frame2=cv2.rectangle(frame1,(x,y),(x+w,y+h),color,2)
        diff_array.append(frame1)
  return dimensions,diff_array

In [101]:
def recognize_bound_for_alphabets(image):
  img=image.copy()
  img_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  img_mean=cv2.adaptiveThreshold(img_gray,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY_INV,11,5)
  sobelX=cv2.Sobel(img_mean,cv2.CV_64F,1,0)
  sobelY=cv2.Sobel(img_mean,cv2.CV_64F,0,1)
  sobelX=np.uint8(np.absolute(sobelX))
  sobelY=np.uint8(np.absolute(sobelY))
  sobelXY=cv2.bitwise_or(sobelX,sobelY)
  contours,hierarchy=cv2.findContours(sobelXY,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  if(len(contours)>0):
    for contour in contours:
      if cv2.contourArea(contour)<300:
        continue
      x,y,w,h=cv2.boundingRect(contour)
      #print(x,y,w,h)
      color=np.random.uniform((255,255,255))
      cv2.drawContours(img,contour,-1,color,5)
      #plt.imshow(img)
      #plt.show()
  #Note: These values are captured  from o/p of above images
  #changes these values based on o/p obtained above
  #values as x,y,w,h
  dict={
     "A":[83,69,282,217],
     "B":[442,78,235,192],
     "C":[734,84,201,177],
     "D":[95,327,285,222],
     "E":[452,323,235,199],
     "F":[742,311,201,183],
   }      
  return dict

In [106]:
def findinside(x1,y1,x2,y2,x,y):
  if((x>x1 and x<x2) and (y>y1 and y<y2)):
    return True
  else:
    return False

In [103]:
def check_is_inside(dict,dimensions,fps):
  output=set()
  for a in dimensions:
    frame_no=a[4]
    x,y,w,h=a[0],a[1],a[2],a[3]
    for k,b in dict.items():
        if(findinside(b[0],b[1],(b[0]+b[2]),(b[1]+b[3]),x,y)):
          output.add((frame_no//fps,k))
          break
  output=list(output)
  #print(output)
  output.sort(key=lambda x:x[0])
  return output

In [108]:
def main_function(video_path):
  frame_array,fps,frame_count,duration=crop_frames(video_path)
  dictionary=recognize_bound_for_alphabets(frame_array[0])
  dimensions,differance_array=diff_btwn_adjacent_frames(frame_array)
  #for x in dimensions:
  #print(x)
  #display_all_frames(differance_array)
  output=check_is_inside(dictionary,dimensions,fps)
  print(output)

In [109]:
if __name__=="__main__":
  videopath="/content/drive/MyDrive/datasets/assignmentVideo.mp4"
  main_function(videopath)

[(1.0, 'A'), (1.0, 'D'), (3.0, 'C'), (6.0, 'E'), (6.0, 'F'), (8.0, 'F')]


In [ ]:
def video_writer(frame_array,fps):
  width=frame_array[1].shape[1]
  height=frame_array[1].shape[0]
  out = cv2.VideoWriter('diff.avi',cv2.VideoWriter_fourcc('M','J','P','G'), fps, (width,height))
  for x in frame_array:
    out.write(x)
  out.release()
  cv2.destroyAllWindows()